# Medicare Part D - Prescription ETL
---
- placeholder text for description
---
### Dependencies and Setup

In [ ]:
# TC


In [ ]:
# EH


In [ ]:
# TMI


In [1]:
# GO
import pandas as pd
import numpy as np
from functools import reduce
import csv


---
## Extraction

In [ ]:
# CMS Part D Prescriber Summary Reports


In [ ]:
# Medicaid.gov - National Average Drug Acquisition Cost


In [2]:
file_csv = "Raw/NADAC__National_Average_Drug_Acquisition_Cost_.csv"

In [3]:
file_csv_df = pd.read_csv(file_csv, encoding="ISO-8859-1")
file_csv_df.head()

,NDC Description,NDC,NADAC_Per_Unit,Effective_Date,Pricing_Unit,Pharmacy_Type_Indicator,OTC,Explanation_Code,Classification_for_Rate_Setting,Corresponding_Generic_Drug_NADAC_Per_Unit,Corresponding_Generic_Drug_Effective_Date,As of Date
0,SIMVASTATIN 80 MG TABLET,16714068502,0.08008,03/23/2016,EA,C/I,N,1,G,NaN,NaN,04/13/2016
1,CARVEDILOL 12.5 MG TABLET,68382009405,0.03281,03/23/2016,EA,C/I,N,1,G,NaN,NaN,04/13/2016
2,CLONAZEPAM 1 MG TABLET,93083310,0.02619,03/23/2016,EA,C/I,N,1,G,NaN,NaN,04/13/2016
3,NAPHCON-A EYE DROPS,65008542,0.56743,12/23/2015,ML,C/I,Y,"4, 5, 6",B,NaN,NaN,04/13/2016
4,OFLOXACIN 0.3% EAR DROPS,24208041005,17.27867,02/17/2016,ML,C/I,N,"4, 5",G,NaN,NaN,04/13/2016


In [4]:
to_drop = ["NDC","Pharmacy_Type_Indicator","Explanation_Code","Corresponding_Generic_Drug_NADAC_Per_Unit","Corresponding_Generic_Drug_Effective_Date","As of Date"]

file_csv_df.drop(to_drop, inplace = True, axis = 1)
file_csv_df.head()

,NDC Description,NADAC_Per_Unit,Effective_Date,Pricing_Unit,OTC,Classification_for_Rate_Setting
0,SIMVASTATIN 80 MG TABLET,0.08008,03/23/2016,EA,N,G
1,CARVEDILOL 12.5 MG TABLET,0.03281,03/23/2016,EA,N,G
2,CLONAZEPAM 1 MG TABLET,0.02619,03/23/2016,EA,N,G
3,NAPHCON-A EYE DROPS,0.56743,12/23/2015,ML,Y,B
4,OFLOXACIN 0.3% EAR DROPS,17.27867,02/17/2016,ML,N,G


In [ ]:
# Q1Medicare
q1_url = 'https://q1medicare.com/PartD-The-2019-Medicare-Part-D-Outlook.php#ME963C921251342D6BE66AB6E656C89F6'


---
## Transformation

In [ ]:
# CMS Clean-up:


In [ ]:
# Medicaid Clean-up:


In [ ]:
# Q1 Clean-up:


---
## Loading